In [ ]:
from flask import Flask, jsonify,request
import pickle
import nltk                                # Natural Language Toolkit library
from nltk.corpus import twitter_samples    # Example Twitter dataset from NLTK
from nltk.corpus import stopwords          # Built-in list of common stopwords
from nltk.stem import PorterStemmer        # Stemmer for reducing words
from nltk.stem import WordNetLemmatizer    # Lemmatizer to get base form of words
from nltk.tokenize import TweetTokenizer
nltk.download('wordnet')
nltk.download('omw-1.4')
import re
import string

app = Flask(__name__)

#*************************************************************************************#
# load pre-trained Logistic Regression model
with open('logistic_regression_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# loading the fitted vectorizer
with open('vectorizer.pkl', 'rb') as file:
    loaded_vectorizer = pickle.load(file)
    
# Custom text preprocessing pipeline
def process_tweet(tweet):
    lemmatizer = WordNetLemmatizer()
    stopwords_english = stopwords.words('english')
    tweet = re.sub(r'\$\w*', '', tweet)          # remove stock market tickers like $AAPL
    tweet = re.sub(r'^RT[\s]+', '', tweet)       # remove retweet tag "RT"
    tweet = re.sub(r'https?://[^\s\n\r]+', '', tweet)    # remove hyperlinks
    tweet = re.sub(r'#', '', tweet)                      # remove hashtag symbol
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  #  ignore stopwords
                word not in string.punctuation):  # ignore punctuation
            lemma_word = lemmatizer.lemmatize(word,pos='v')  # lemmatize word as verb
            tweets_clean.append(lemma_word)

    return tweets_clean



# convert list of words back to string
def list_to_string(lst):
    return ' '.join(lst)
#****************************************************************************************#

@app.route('/prediction', methods=['POST'])
def predicting_sentiment():

    # get input JSON payload
    data = request.get_json()

    input_string = data['input_string']

    # step 1: text preprocessing
    tokenize_data = process_tweet(input_string)

   # step 2: join tokens into single string
    final_string = list_to_string(tokenize_data)

   # step 3: convert text into vector form
    text_vector = loaded_vectorizer.transform([final_string])

    # step 4: perform sentiment prediction
    prediction = loaded_model.predict(text_vector)
    sentiment_mapping = {0: 'Negative', 1: 'Positive'}
    predicted_sentiment = sentiment_mapping[prediction[0]]

    return jsonify({'result': predicted_sentiment}), 200



@app.route('/status', methods=['GET'])
def status():
    return jsonify({'message': 'working NOW'}), 200



if __name__ == '__main__':
    app.run(debug=True)